# Análisis Exploratorio de Datos - Clasificación de Literatura Médica

Este notebook contiene el análisis exploratorio del dataset de literatura médica para el Tech Sphere Challenge. El objetivo es entender la estructura de los datos, las características principales y preparar el terreno para el desarrollo de modelos de clasificación multi-etiqueta.

## Importación de librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter

# Configuraciones
plt.style.use('ggplot')
sns.set(style='whitegrid')
pd.set_option('display.max_columns', None)

# Descargar recursos de NLTK
nltk.download('punkt')
nltk.download('stopwords')

## Carga de datos

In [ ]:
# Cargar el dataset
data_path = '../data/raw/challenge_data-18-ago.csv'
df = pd.read_csv(data_path, sep=';')

# Mostrar las primeras filas
print(f'Dimensiones del dataset: {df.shape}')
df.head()

## Información general del dataset

In [ ]:
# Información general
df.info()

# Estadísticas descriptivas
df.describe(include='all')

In [ ]:
# Verificar valores nulos
print('Valores nulos por columna:')
print(df.isnull().sum())

# Verificar valores duplicados
print(f'
Número de filas duplicadas: {df.duplicated().sum()}')

## Análisis de las etiquetas (grupos)

In [ ]:
# Analizar la columna 'group'
print('Valores únicos en la columna group:')
print(df['group'].unique())

# Separar las etiquetas múltiples (multi-label)
def extract_labels(group_str):
    if pd.isna(group_str):
        return []
    return group_str.split('|')

# Extraer todas las etiquetas
all_labels = df['group'].apply(extract_labels)

# Contar la frecuencia de cada etiqueta
label_counts = Counter([label for labels in all_labels for label in labels])

# Mostrar la distribución de etiquetas
print('
Distribución de etiquetas:')
for label, count in label_counts.items():
    print(f'{label}: {count}')

In [ ]:
# Visualizar la distribución de etiquetas
plt.figure(figsize=(10, 6))
sns.barplot(x=list(label_counts.keys()), y=list(label_counts.values()))
plt.title('Distribución de Etiquetas')
plt.xlabel('Categoría')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Analizar la cantidad de etiquetas por artículo
label_count_per_article = all_labels.apply(len)

print('Estadísticas de etiquetas por artículo:')
print(label_count_per_article.describe())

# Visualizar la distribución de cantidad de etiquetas por artículo
plt.figure(figsize=(8, 5))
sns.countplot(x=label_count_per_article)
plt.title('Número de Etiquetas por Artículo')
plt.xlabel('Cantidad de Etiquetas')
plt.ylabel('Frecuencia')
plt.show()

## Análisis de los títulos

In [ ]:
# Estadísticas de longitud de los títulos
df['title_length'] = df['title'].apply(len)

print('Estadísticas de longitud de títulos:')
print(df['title_length'].describe())

# Visualizar la distribución de longitud de títulos
plt.figure(figsize=(10, 6))
sns.histplot(df['title_length'], bins=30, kde=True)
plt.title('Distribución de Longitud de Títulos')
plt.xlabel('Longitud (caracteres)')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Análisis de palabras en los títulos
def preprocess_text(text):
    if pd.isna(text):
        return []
    # Convertir a minúsculas y eliminar caracteres especiales
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Tokenizar
    tokens = word_tokenize(text)
    # Eliminar stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

# Procesar títulos
title_tokens = df['title'].apply(preprocess_text)

# Contar frecuencia de palabras en títulos
title_word_counts = Counter([word for tokens in title_tokens for word in tokens])

# Mostrar las palabras más comunes en los títulos
print('Palabras más comunes en los títulos:')
for word, count in title_word_counts.most_common(20):
    print(f'{word}: {count}')

In [ ]:
# Visualizar las palabras más comunes en los títulos
plt.figure(figsize=(12, 8))
words = [word for word, count in title_word_counts.most_common(20)]
counts = [count for word, count in title_word_counts.most_common(20)]

sns.barplot(x=counts, y=words)
plt.title('20 Palabras Más Comunes en los Títulos')
plt.xlabel('Frecuencia')
plt.ylabel('Palabra')
plt.tight_layout()
plt.show()

## Análisis de los abstracts

In [ ]:
# Estadísticas de longitud de los abstracts
df['abstract_length'] = df['abstract'].apply(lambda x: len(x) if pd.notna(x) else 0)

print('Estadísticas de longitud de abstracts:')
print(df['abstract_length'].describe())

# Visualizar la distribución de longitud de abstracts
plt.figure(figsize=(10, 6))
sns.histplot(df['abstract_length'], bins=30, kde=True)
plt.title('Distribución de Longitud de Abstracts')
plt.xlabel('Longitud (caracteres)')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# Análisis de palabras en los abstracts
abstract_tokens = df['abstract'].apply(preprocess_text)

# Contar frecuencia de palabras en abstracts
abstract_word_counts = Counter([word for tokens in abstract_tokens for word in tokens])

# Mostrar las palabras más comunes en los abstracts
print('Palabras más comunes en los abstracts:')
for word, count in abstract_word_counts.most_common(20):
    print(f'{word}: {count}')

In [ ]:
# Visualizar las palabras más comunes en los abstracts
plt.figure(figsize=(12, 8))
words = [word for word, count in abstract_word_counts.most_common(20)]
counts = [count for word, count in abstract_word_counts.most_common(20)]

sns.barplot(x=counts, y=words)
plt.title('20 Palabras Más Comunes en los Abstracts')
plt.xlabel('Frecuencia')
plt.ylabel('Palabra')
plt.tight_layout()
plt.show()

## Análisis por categoría

In [ ]:
# Crear columnas binarias para cada categoría
categories = ['cardiovascular', 'hepatorenal', 'neurological', 'oncological']

for category in categories:
    df[category] = df['group'].apply(lambda x: 1 if pd.notna(x) and category in x else 0)

# Mostrar la distribución de categorías
print('Distribución de categorías:')
for category in categories:
    print(f'{category}: {df[category].sum()}')

In [ ]:
# Visualizar la distribución de categorías
plt.figure(figsize=(10, 6))
category_counts = [df[category].sum() for category in categories]
sns.barplot(x=categories, y=category_counts)
plt.title('Distribución de Categorías')
plt.xlabel('Categoría')
plt.ylabel('Frecuencia')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Analizar co-ocurrencia de categorías
cooccurrence = pd.DataFrame()

for cat1 in categories:
    for cat2 in categories:
        if cat1 != cat2:
            cooccurrence.loc[cat1, cat2] = ((df[cat1] == 1) & (df[cat2] == 1)).sum()
        else:
            cooccurrence.loc[cat1, cat2] = df[cat1].sum()

# Visualizar matriz de co-ocurrencia
plt.figure(figsize=(10, 8))
sns.heatmap(cooccurrence, annot=True, fmt='d', cmap='YlGnBu')
plt.title('Matriz de Co-ocurrencia de Categorías')
plt.tight_layout()
plt.show()

## Análisis de palabras por categoría

In [ ]:
# Palabras más comunes por categoría
for category in categories:
    # Filtrar artículos de esta categoría
    category_df = df[df[category] == 1]
    
    # Procesar títulos y abstracts
    cat_title_tokens = category_df['title'].apply(preprocess_text)
    cat_abstract_tokens = category_df['abstract'].apply(preprocess_text)
    
    # Contar palabras
    cat_title_word_counts = Counter([word for tokens in cat_title_tokens for word in tokens])
    cat_abstract_word_counts = Counter([word for tokens in cat_abstract_tokens for word in tokens])
    
    print(f'
Palabras más comunes en títulos de la categoría {category}:')
    for word, count in cat_title_word_counts.most_common(10):
        print(f'{word}: {count}')
    
    print(f'
Palabras más comunes en abstracts de la categoría {category}:')
    for word, count in cat_abstract_word_counts.most_common(10):
        print(f'{word}: {count}')

## Longitud de texto por categoría

In [ ]:
# Comparar longitud de títulos y abstracts por categoría
plt.figure(figsize=(12, 6))

# Longitud de títulos por categoría
plt.subplot(1, 2, 1)
data = []
labels = []

for category in categories:
    category_df = df[df[category] == 1]
    data.append(category_df['title_length'])
    labels.append(category)

plt.boxplot(data, labels=labels)
plt.title('Longitud de Títulos por Categoría')
plt.ylabel('Longitud (caracteres)')
plt.xticks(rotation=45)

# Longitud de abstracts por categoría
plt.subplot(1, 2, 2)
data = []
labels = []

for category in categories:
    category_df = df[df[category] == 1]
    data.append(category_df['abstract_length'])
    labels.append(category)

plt.boxplot(data, labels=labels)
plt.title('Longitud de Abstracts por Categoría')
plt.ylabel('Longitud (caracteres)')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## Conclusiones del análisis exploratorio

Después de realizar el análisis exploratorio de los datos, podemos concluir lo siguiente:

1. **Estructura del dataset**:
   - El dataset contiene 3,565 registros con 3 columnas: título, abstract y grupo (categoría).
   - Las categorías son: cardiovascular, hepatorenal, neurological y oncological.
   - Es un problema de clasificación multi-etiqueta, donde un artículo puede pertenecer a más de una categoría.

2. **Distribución de categorías**:
   - [Completar con los resultados del análisis]
   - La co-ocurrencia de categorías muestra que [completar con los resultados].

3. **Características de los textos**:
   - Los títulos tienen una longitud media de [completar] caracteres.
   - Los abstracts tienen una longitud media de [completar] caracteres.
   - Las palabras más comunes en los títulos son [completar].
   - Las palabras más comunes en los abstracts son [completar].

4. **Diferencias entre categorías**:
   - [Completar con las diferencias observadas entre categorías]

5. **Próximos pasos**:
   - Preprocesamiento de texto: limpieza, tokenización, eliminación de stopwords, stemming/lemmatización.
   - Ingeniería de características: TF-IDF, word embeddings, características específicas del dominio médico.
   - Selección de modelos: probar diferentes enfoques para clasificación multi-etiqueta.
   - Evaluación: utilizar métricas adecuadas para problemas multi-etiqueta, como F1 score ponderado.

In [ ]:
# Guardar el DataFrame con las características adicionales
df.to_csv('../data/processed/medical_literature_with_features.csv', index=False)